In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
create schema if not exists retail_analytics.bronze

In [0]:
cust_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/retail_analytics/raw/kaggle/global_fashion/customers.csv")

In [0]:
display(cust_df.limit(5))

In [0]:
import re

In [0]:
# Sanitize column names (MANDATORY for Bronze)
clean_columns = [re.sub(r'[ ,;{}()\n\t=]', '_', c) for c in cust_df.columns]
cust_df = cust_df.toDF(*clean_columns)

# Add ingestion timestamp
cust_df = cust_df.withColumn("ingestion_ts", current_timestamp())

In [0]:
(cust_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("retail_analytics.bronze.customers")
)

In [0]:
spark.read.table("retail_analytics.bronze.customers").limit(5).display()